In [209]:
from google.cloud import storage

In [210]:
storage_client = storage.Client.from_service_account_json('../service_account.json')

In [ ]:
import json
bucket_names = json.load("../bucket_names.json")
csv_bucket = bucket_names["csv_bucket"]

In [211]:
# Tables to check
for blob in storage_client.list_blobs(csv_bucket):
    print(blob.name, '\tSize:', blob.size)

Copia de Copia de T-Cotizaciones.csv 	Size: 927151
Copia de T-CotizacionesOriginal.csv 	Size: 927151
Errores de pegado.csv 	Size: 5956
Paste Errors.csv 	Size: 28246
T- % IVA.csv 	Size: 55
T-Abonos Main.csv 	Size: 389780
T-Abonos.csv 	Size: 311811
T-Abonos1.csv 	Size: 229618
T-AbonosCredito.csv 	Size: 187
T-AbonosCredito1.csv 	Size: 187
T-Almacenes.csv 	Size: 115
T-BarcodesCentralStore4.csv 	Size: 269
T-Cierres.csv 	Size: 32813
T-Ciudades.csv 	Size: 1102
T-CodesStore4B4TransferByPhone.csv 	Size: 358526
T-Codigos de barras nosirve.csv 	Size: 29817800
T-Codigos de barras.csv 	Size: 41106104
T-Codigos de barras2.csv 	Size: 32557909
T-CodigosDeBarrasB4TransferByPhone.csv 	Size: 41078864
T-CodigosDeBarrasFromCentral.csv 	Size: 43414795
T-CodigosDeBarrasFromGC.csv 	Size: 27892
T-Cotizaciones.csv 	Size: 45918
T-Cotizaciones2.csv 	Size: 7373666
T-Cotizaciones2Original.csv 	Size: 6426214
T-CotizacionesOriginal.csv 	Size: 2806882
T-Credito Atraso.csv 	Size: 0
T-Credito.csv 	Size: 219
T-CreditosAm

## Facts:
- [x] T-Codigos de barras.csv
- [x] T-Cotizaciones.csv - T-Cotizaciones2.csv - T-Cotizaciones2Original.csv - T-CotizacionesOriginal.csv (La union de ambos es la Fact. Revisar que no hallan duplicados)
- [x] T-Datos cliente.csv
- [x] T-Factura.csv - T-FacturaOriginal.csv (La union de ambos es la Fact. Revisar que no hallan duplicados)
- [x] T-Factura cotizacion.csv - T-Factura cotizacion2.csv (La union de ambos es la Fact. Revisar que no hallan duplicados)
- [x] T-Formas de pago Cotizaciones.csv - T-Formas de pago.csv (La union de ambos es la Fact. Revisar que no hallan duplicados)
- [x] T-Inventario dañado.csv
- [x] T-Inventario.csv
- [x] T-Ventas.csv - T-Ventas1.csv

## Dims:
- [x] T-Lista colores.csv
- [x] T-Lista siluetas.csv
- [x] T-Lista tallas.csv
- [x] T-Sexo.csv
- [x] T-Tipo de prenda.csv

## Notas:
- Cotizaciones y ventas son lo mismo

In [212]:
def get_csv(filename):
    target_path = '../data/' + filename
    file_bucket = storage_client.bucket(csv_bucket)
    file_blob = storage.Blob(filename, file_bucket)
    with open(target_path, 'wb') as file_handle:
        file_blob.download_to_file(file_handle)

In [213]:
get_csv('T-Codigos de barras.csv')

In [214]:
import pandas as pd

In [215]:
barcodes = pd.read_csv('../data/T-Codigos de barras.csv')
barcodes = barcodes[['Codigo de barras', 'Referencia', 'Tipo prenda', 'Sexo', 'Silueta', 'Color', 'Talla']]
barcodes = barcodes.rename(columns = { 'Codigo de barras': 'barcode', 'Referencia': 'reference', 'Tipo prenda': 'clothing_type_id', 'Sexo': 'gender_id', 'Silueta': 'silhouette_id', 'Color':  'color_id', 'Talla': 'size_id' })
# Save data
barcodes.reset_index(drop = True).to_pickle('../data/processed/barcodes.pkl')
# Show result
barcodes.head()

,barcode,reference,clothing_type_id,gender_id,silhouette_id,color_id,size_id
0,0175440,NaN,0.0,NaN,NaN,NaN,NaN
1,0328308,P40568,0.0,NaN,NaN,NaN,NaN
2,0334170,DAÑADO,0.0,NaN,NaN,NaN,NaN
3,0633818,001,NaN,NaN,NaN,NaN,06
4,10011011001300M322,C02309,101.0,1.0,100.0,130.0,0M


In [216]:
get_csv('T-Lista colores.csv')

In [217]:
colors = pd.read_csv('../data/T-Lista colores.csv')
colors = colors.rename(columns = { 'Color': 'description', '#Color': 'id' })
# Save data
colors.reset_index(drop = True).to_pickle('../data/processed/colors.pkl')
# Show result
colors.head()

,description,id
0,INDIGO CON CAMUFLADO,749
1,INDIGO CON RAYAS AZUL,750
2,INDIGO CON RAYAS CAMEL,751
3,INDIGO CON RAYAS ROSADO,752
4,CIRCULOS BLANCO,753


In [218]:
get_csv('T-Tipo de prenda.csv')

In [219]:
clothing_types = pd.read_csv('../data/T-Tipo de prenda.csv')
clothing_types = clothing_types.rename(columns = { 'Tipo de prenda': 'description', 'id tipo de prenda': 'id' })
# Save data
clothing_types.reset_index(drop = True).to_pickle('../data/processed/clothing_types.pkl')
# Show result
clothing_types.head()

,description,id
0,BLUSA,100
1,CAMISA,101
2,CAMISETA,102
3,BUZO,103
4,CAMISUETER,104


In [220]:
get_csv('T-Sexo.csv')

In [221]:
genders = pd.read_csv('../data/T-Sexo.csv')
genders = genders.rename(columns = { 'Sexo': 'description', 'Id sexo': 'id' })
# Save data
genders.reset_index(drop = True).to_pickle('../data/processed/genders.pkl')
# Show result
genders.head()

,description,id
0,MASC,1
1,FEM,2
2,UNISEX,5
3,JUNIOR,6


In [222]:
get_csv('T-Lista siluetas.csv')

In [223]:
silhouettes = pd.read_csv('../data/T-Lista siluetas.csv')
silhouettes = silhouettes.rename(columns = { 'Silueta': 'description', 'id silueta': 'id' })
# Save data
silhouettes.reset_index(drop = True).to_pickle('../data/processed/silhouettes.pkl')
# Show result
silhouettes.head()

,description,id
0,TOP TIRAS,596
1,ILLUSIONATOR,267
2,GAZELLE,268
3,LOW ZENERATION,269
4,NORMAL BO ANGOST,118


In [224]:
get_csv('T-Lista tallas.csv')

In [225]:
sizes = pd.read_csv('../data/T-Lista tallas.csv')
sizes = sizes.rename(columns = { 'Talla': 'id' })
# Save data
sizes.reset_index(drop = True).to_pickle('../data/processed/sizes.pkl')
# Show result
sizes.head()

,id
0,01
1,02
2,03
3,04
4,05


In [226]:
get_csv('T-Datos cliente.csv')

In [227]:
import re
from datetime import datetime
email_regex = '^[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}$'
birthday_regex = '^\d\d/\d\d/\d\d\d\d$'
birthday_format = '%d/%m/%Y'

In [228]:
clients = pd.read_csv('../data/T-Datos cliente.csv')
clients['Nombre'] = clients['Nombre'].str.upper() + ' ' +  clients['Segundo apellido'].str.upper()
clients['Email'] = clients['Referencia'].fillna('').apply(lambda x: x if re.search(email_regex, x) else '')
clients['Nombre'] = clients['Nombre'].fillna('')
clients['Direccion'] = clients['Direccion'].fillna('')
clients['Ciudad'] = clients['Ciudad'].fillna('')
clients['Telefono casa num'] = clients['Telefono casa num'].fillna(0).astype('int64')
clients['Telefono casa'] = clients['Telefono casa'].astype('float').fillna(0).apply(lambda x: x if x < 10**10 else 0).astype('int64')
clients['Telefono casa'] = clients.apply(lambda row: row['Telefono casa'] if row['Telefono casa'] != 0 else row['Telefono casa num'], axis = 1)
clients['Telefono casa'] = clients['Telefono casa']
clients['Telefono trabajo'] = clients['Telefono trabajo'].fillna(0).astype('int64')
clients['Telefono celular'] = clients['Telefono celular'].fillna(0).astype('int64')
clients['Barrio'] = clients['Barrio'].fillna('')
clients['Cumpleaños'] = clients['Cumpleaños'].fillna('').apply(lambda x: datetime.strptime(x, birthday_format) if re.search(birthday_regex, x) else datetime.strptime('1900-01-01', '%Y-%m-%d'))
clients = clients[['Nombre', 'Cedula', 'Direccion', 'Ciudad', 'Email',
    'Telefono trabajo', 'Telefono celular', 'Barrio', 'Cumpleaños', 'Telefono casa']]
clients = clients.rename(columns = { 'Nombre': 'name', 'Cedula': 'id', 'Direccion': 'address',
    'Ciudad': 'city', 'Telefono casa num': 'home_phone', 'Email': 'email', 'Telefono trabajo': 'work_phone',
    'Telefono celular': 'cellphone', 'Barrio': 'neighborhood', 'Cumpleaños': 'birthday',
    'Telefono casa': 'home_phone' })
# Save data
clients.reset_index(drop = True).to_pickle('../data/processed/clients.pkl')
# Show result
clients.head()

,name,id,address,city,email,work_phone,cellphone,neighborhood,birthday,home_phone
0,,1.000000,,BARRANQUILLA,,0,0,,1900-01-01,0
1,,1.558014,,BARRANQUILLA,,0,0,,1900-01-01,0
2,,2.000000,Barranquilla,,,0,0,,1900-01-01,3780368
3,,2.004005,WINSOR,BARRANQUILLA,,0,0,,1900-01-01,0
4,,3.000000,,,,0,0,,1900-01-01,0


In [229]:
get_csv('T-Factura.csv')
get_csv('T-FacturaOriginal.csv')
get_csv('T-Factura cotizacion.csv')
get_csv('T-Factura cotizacion2.csv')

In [230]:
# Venta
columns = ['Factura #', 'Fecha Venta', 'Vendedora', 'Cedula', 'hora']
mapping = { 'Factura #': 'id', 'Fecha Venta': 'date',  'Vendedora': 'salesperson_id', 'Cedula': 'client_id', 'hora': 'time' }
invoices_1 = pd.read_csv('../data/T-Factura.csv')
invoices_2 = pd.read_csv('../data/T-FacturaOriginal.csv')
invoices_1 = invoices_1[columns]
invoices_2 = invoices_2[columns]
invoices_1 = invoices_1.rename(columns = mapping)
invoices_2 = invoices_2.rename(columns = mapping)
# Cotizaciones
columns = ['cFactura #', 'Fecha Venta', 'Vendedora', 'Cedula', 'hora']
mapping = { 'cFactura #': 'id', 'Fecha Venta': 'date', 'Vendedora': 'salesperson_id', 'Cedula': 'client_id', 'hora': 'time' }
invoices_3 = pd.read_csv('../data/T-Factura cotizacion.csv')
invoices_4 = pd.read_csv('../data/T-Factura cotizacion2.csv')
invoices_3 = invoices_3[columns]
invoices_4 = invoices_4[columns]
invoices_3 = invoices_3.rename(columns = mapping)
invoices_4 = invoices_4.rename(columns = mapping)
# Join
invoices = pd.concat([invoices_1, invoices_2, invoices_3, invoices_4])
invoices['id'] = invoices['id'].astype('int64')
invoices['client_id'] = invoices['client_id'].fillna(0).astype('int64')
# Clean date
invoices['date'] = pd.to_datetime(invoices['date'], format = '%m/%d/%y 00:00:00')
# Clean time
dirty_times = invoices['time'].notnull() \
    & ~invoices['time'].fillna('').str.endswith('p.m.') \
    & ~invoices['time'].fillna('').str.endswith('a.m.') \
    & ~invoices['time'].fillna('').str.match('^\d\d:\d\d$') \
    & ~invoices['time'].fillna('').str.match('^\d\d:\d\d $') \
    & ~invoices['time'].fillna('').str.endswith('a. m.') \
    & ~invoices['time'].fillna('').str.endswith('p. m.')
invoices.loc[dirty_times, 'time'] = None
invoices['time'] = invoices['time'].fillna('12:00 AM')
invoices['time'] = invoices['time'].str.replace('a.m.', 'AM')
invoices['time'] = invoices['time'].str.replace('p.m.', 'PM')
invoices['time'] = invoices['time'].str.replace('a. m.', 'AM')
invoices['time'] = invoices['time'].str.replace('p. m.', 'PM')
def add_period(x):
    x = x.strip()
    if not x.endswith('AM') and not x.endswith('PM'):
        v = int(x[0:2])
        return x + ' AM' if v < 12 else x + ' PM'
    else:
        return x
invoices['time'] = invoices['time'].apply(add_period)
invoices['time'] = invoices['time'].apply(lambda x: datetime.strptime(x, '%I:%M %p'))
invoices['time'] = invoices['time'].apply(lambda x: x.strftime('%H:%M:00'))
# Save data
invoices.reset_index(drop = True).to_pickle('../data/processed/invoices.pkl')
# Show result
invoices.head()

,id,date,salesperson_id,client_id,time
0,1,2017-05-23,MARIA CLAUDIA,68303171,19:09:00
1,2,2017-05-23,MARIA CLAUDIA,63298383,19:32:00
2,3,2017-05-24,MARIA CLAUDIA,40992669,09:48:00
3,4,2017-05-24,MARIA CLAUDIA,32796794,19:10:00
4,5,2017-05-25,MARIA CLAUDIA,45582096,10:23:00


In [231]:
get_csv('T-Inventario.csv')
get_csv('T-Inventario dañado.csv')

In [232]:
inventory = pd.read_csv('../data/T-Inventario.csv')
inventory = inventory[['Codigo de barras', 'Inventario']]
inventory = inventory.rename(columns = {'Codigo de barras': 'barcode', 'Inventario': 'quantity' })
inventory = inventory[inventory['quantity'] > 0]
# Save data
inventory.reset_index(drop = True).to_pickle('../data/processed/inventory.pkl')
# Show result
inventory.head()

,barcode,quantity
91361,179211921110108199012,2.0
108838,182210010010070S340271,1.0
108910,18221001001010M340026,3.0
108967,18221001001010S340025,1.0
108997,182210010010170S340274,2.0


In [233]:
damaged = pd.read_csv('../data/T-Inventario dañado.csv')
damaged = damaged.rename(columns = { 'Codigo de barras': 'barcode', 'Dañado': 'quantity' })
# Save data
damaged.reset_index(drop = True).to_pickle('../data/processed/damaged.pkl')
# Show result
damaged.head()

,barcode,quantity
0,17921001001860M71773,1
1,17921001012910M73459,1
2,18021001002590M78179,1
3,17921001008150M71578,1
4,17921003971050M78252,1


In [234]:
get_csv('T-Ventas.csv')
get_csv('T-Ventas1.csv')
get_csv('T-Cotizaciones.csv')
get_csv('T-CotizacionesOriginal.csv')

In [235]:
columns = ['# factura', 'Fecha venta', 'Codigo de barras', '% Descuento', 'Descuento', 'Precio venta fijo']
mapping = { '# factura': 'invoice_id', 'Fecha venta': 'date', 'Codigo de barras': 'barcode', '% Descuento': 'discount_percentage', 'Descuento': 'discount', 'Precio venta fijo': 'price' }
# Read CSVs
sales_1 = pd.read_csv('../data/T-Ventas.csv')
sales_2 = pd.read_csv('../data/T-Ventas1.csv')
sales_3 = pd.read_csv('../data/T-Cotizaciones.csv')
sales_4 = pd.read_csv('../data/T-CotizacionesOriginal.csv')
# Get Columns
sales_1 = sales_1[columns]
sales_2 = sales_2[columns]
sales_3['Precio venta fijo'] = sales_3['Precio venta Fijo']
sales_3 = sales_3[columns]
sales_4['Precio venta fijo'] = sales_4['Precio venta Fijo']
sales_4 = sales_4[columns]
# Rename columns
sales_1 = sales_1.rename(columns = mapping)
sales_2 = sales_2.rename(columns = mapping)
sales_3 = sales_3.rename(columns = mapping)
sales_4 = sales_4.rename(columns = mapping)
# Join parts
sales = pd.concat([sales_1, sales_2, sales_3, sales_4])
sales = sales.reset_index(drop = True)
# Clean barcodes
sales = sales[sales['barcode'].notnull()]
# Clean sales price
sales = sales[sales['price'].notnull()]
# Clean discount_percentage
sales['discount_percentage'] = sales['discount_percentage'].fillna(0)
sales['discount_percentage'] = sales['discount_percentage'].apply(lambda x: x / 100.0 if x >= 1 else x)
# Clean discount
sales['discount'] = sales['discount'].fillna(0)
# Merge discount_percentage and discount
tmp1 = sales[sales['discount'] != 0].groupby('invoice_id')[['price']].sum()
tmp2 = sales[sales['discount'] != 0].groupby('invoice_id')[['discount']].max()
sales['discount_percentage_2'] = tmp2['discount'] / tmp1['price']
sales['discount_percentage_2'] = sales['discount_percentage_2'].fillna(0)
sales['discount_percentage'] = sales.apply(lambda row: row['discount_percentage'] if row['discount_percentage'] != 0 else row['discount_percentage_2'], axis = 1)
sales = sales.drop(['discount', 'discount_percentage_2'], axis = 1)
sales = sales[sales['invoice_id'].notnull()]
sales['invoice_id'] = sales['invoice_id'].astype('int64')
sales['price'] = sales['price'].astype('int64')
# Clean date
sales['date'] = pd.to_datetime(sales['date'], format = '%m/%d/%y 00:00:00')
# Save data
sales.reset_index(drop = True).to_pickle('../data/processed/sales.pkl')
# Show result
sales.head()

,invoice_id,date,barcode,discount_percentage,price
0,20901,2020-01-25,182210985123210M342923,0.1,299900
1,20901,2020-01-25,18221008571010M341755,0.1,69900
2,20901,2020-01-25,20121008001600S332633,0.1,60000
3,20901,2020-01-25,18221004551090S341913,0.1,69900
4,20901,2020-01-25,18221008571110S341760,0.1,69900


In [236]:
get_csv('T-Formas de pago.csv')
get_csv('T-Formas de pago Cotizaciones.csv')

In [237]:
# Read CSVs
payments_1 = pd.read_csv('../data/T-Formas de pago.csv')
payments_1 = payments_1[['Factura #', 'Fecha', 'Tipo de pago', 'Cantidad']]
payments_1 = payments_1.rename(columns = { 'Factura #': 'invoice_id', 'Fecha': 'date', 'Tipo de pago': 'type', 'Cantidad': 'amount' })

payments_2 = pd.read_csv('../data/T-Formas de pago Cotizaciones.csv')
payments_2 = payments_2[['Cfactura #', 'Fecha', 'Tipo de pago', 'Cantidad']]
payments_2 = payments_2.rename(columns = { 'Cfactura #': 'invoice_id', 'Fecha': 'date', 'Tipo de pago': 'type', 'Cantidad': 'amount' })

payments = pd.concat([payments_1, payments_2])
# Clean data
payments['date'] = pd.to_datetime(payments['date'], format = '%m/%d/%y 00:00:00')
payments['invoice_id'] = payments['invoice_id'].astype('int64')
payments = payments[payments['amount'].notnull()]
payments['amount'] = payments['amount'].astype('int64')
# Save data
payments.reset_index(drop = True).to_pickle('../data/processed/payments.pkl')
# Show result
payments.head()

,invoice_id,date,type,amount
0,20901,2020-01-25,Efectivo,566000
1,20902,2020-01-25,T. Debito,350000
2,20903,2020-01-25,Master card,719600
3,20904,2020-01-25,Efectivo,152900
4,20905,2020-01-27,Amex,79900
